In [64]:
#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        np.random.seed(42)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [65]:
test = Network([2,6,4,3])

In [66]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data[:,[0,2]].T
y = data.target
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
Y = lb.fit_transform(y)
Y = Y.T

In [67]:
for array in test.biases:
    print(array.shape)
    print(array)

(6, 1)
[[ 0.49671415]
 [-0.1382643 ]
 [ 0.64768854]
 [ 1.52302986]
 [-0.23415337]
 [-0.23413696]]
(4, 1)
[[ 1.57921282]
 [ 0.76743473]
 [-0.46947439]
 [ 0.54256004]]
(3, 1)
[[-0.46341769]
 [-0.46572975]
 [ 0.24196227]]


In [70]:
for array in test.weights:
    print(array.shape)
    print(array)

(6, 2)
[[-1.91328024 -1.72491783]
 [-0.56228753 -1.01283112]
 [ 0.31424733 -0.90802408]
 [-1.4123037   1.46564877]
 [-0.2257763   0.0675282 ]
 [-1.42474819 -0.54438272]]
(4, 6)
[[ 0.11092259 -1.15099358  0.37569802 -0.60063869 -0.29169375 -0.60170661]
 [ 1.85227818 -0.01349722 -1.05771093  0.82254491 -1.22084365  0.2088636 ]
 [-1.95967012 -1.32818605  0.19686124  0.73846658  0.17136828 -0.11564828]
 [-0.3011037  -1.47852199 -0.71984421 -0.46063877  1.05712223  0.34361829]]
(3, 4)
[[-1.76304016  0.32408397 -0.38508228 -0.676922  ]
 [ 0.61167629  1.03099952  0.93128012 -0.83921752]
 [-0.30921238  0.33126343  0.97554513 -0.47917424]]


In [50]:
test.feedforward(X)

array([[0.08582969, 0.08595973, 0.08604306, 0.08638807, 0.08588923,
        0.08594901, 0.08625181, 0.08598072, 0.08653143, 0.08606034,
        0.08577022, 0.08629021, 0.0860426 , 0.08635461, 0.0855661 ,
        0.08569462, 0.08565528, 0.08582969, 0.08583988, 0.08591321,
        0.08594901, 0.08591321, 0.08598363, 0.08614133, 0.08686833,
        0.08609393, 0.08609393, 0.08585642, 0.08577986, 0.08641288,
        0.08629021, 0.08577022, 0.08585642, 0.08567816, 0.08606034,
        0.08576798, 0.08563199, 0.08595973, 0.08639685, 0.08591321,
        0.08581855, 0.08626162, 0.08639685, 0.08609393, 0.08645812,
        0.0860426 , 0.08601677, 0.08625181, 0.08580912, 0.08588923,
        0.09190239, 0.09349217, 0.09346672, 0.09524377, 0.09362075,
        0.09807533, 0.09554364, 0.09353621, 0.09310031, 0.09649394,
        0.09449498, 0.09424024, 0.09230269, 0.09690267, 0.09173211,
        0.091534  , 0.09880805, 0.09409957, 0.09464853, 0.0938121 ,
        0.09920292, 0.09183205, 0.09715789, 0.09